In [55]:
import pygame
import sys


# Constants
WIDTH, HEIGHT = 600, 600
GRID_SIZE = 8
CELL_SIZE = WIDTH // GRID_SIZE
BLACK = (0, 0, 0)
WHITE = (255, 255, 255)
GREEN = (0, 128, 0)


# Initialize the game board
board = [[0 for _ in range(GRID_SIZE)] for _ in range(GRID_SIZE)]
board[3][4] = board[4][3] = 2  # White
board[3][3] = board[4][4] = 1  # Black

# Current player (1 for White, 2 for Black)


# Initialize Pygame
# # Create the game window

pygame.init()
pygame.font.init() 
my_font = pygame.font.SysFont('Comic Sans MS', 30)

screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("6x6 Othello")

def draw_board():
    screen.fill(GREEN)
    for x in range(GRID_SIZE):
        for y in range(GRID_SIZE):
            pygame.draw.rect(screen, BLACK, (x*CELL_SIZE, y*CELL_SIZE, CELL_SIZE, CELL_SIZE), 1)
            if board[y][x] == 1:
                pygame.draw.circle(screen, WHITE, (x*CELL_SIZE + CELL_SIZE//2, y*CELL_SIZE + CELL_SIZE//2), CELL_SIZE//2 - 5)
            elif board[y][x] == 2:
                pygame.draw.circle(screen, BLACK, (x*CELL_SIZE + CELL_SIZE//2, y*CELL_SIZE + CELL_SIZE//2), CELL_SIZE//2 - 5)

def is_valid_move(x, y,board,current_player):
    if board[y][x] != 0:
        return False
    directions = [(0,1),(1,1),(1,0),(1,-1),(0,-1),(-1,-1),(-1,0),(-1,1)]
    for dx, dy in directions:
        if check_direction(x, y, dx, dy,board,current_player):
            return True
    return False

def check_direction(x, y, dx, dy,board,current_player):
    opponent = 1 if current_player == 2 else 2
    x, y = x + dx, y + dy
    if not (0 <= x < GRID_SIZE and 0 <= y < GRID_SIZE) or board[y][x] != opponent:
        return False
    while 0 <= x < GRID_SIZE and 0 <= y < GRID_SIZE:
        if board[y][x] == 0:
            return False
        if board[y][x] == current_player:
            return True
        x, y = x + dx, y + dy
    return False

def make_move(x, y,board,current_player):
    board[y][x] = current_player
    directions = [(0,1),(1,1),(1,0),(1,-1),(0,-1),(-1,-1),(-1,0),(-1,1)]
    for dx, dy in directions:
        if check_direction(x, y, dx, dy,board,current_player):
            flip_direction(x, y, dx, dy,board,current_player)

def flip_direction(x, y, dx, dy,board,current_player):
    x, y = x + dx, y + dy
    while board[y][x] != current_player:
        board[y][x] = current_player
        x, y = x + dx, y + dy






In [56]:
import torch 
from torch import nn
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(64, 32),
            nn.LeakyReLU(),
            nn.Linear(32,16),
            nn.LeakyReLU(),
            nn.Linear(16, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits
 
device = "cuda"
OthelloNet = NeuralNetwork().to(device)
learning_rate = 1e-3 * 5
loss_fn = nn.MSELoss()
optimizer = torch.optim.AdamW(OthelloNet.parameters(), lr=learning_rate)

OthelloNet.load_state_dict(torch.load('model1.pth'))

C:\Users\ayush\AppData\Local\Temp\ipykernel_7768\3963318844.py:25: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  OthelloNet.load_state_dict(torch.load('model1.pth'))


<All keys matched successfully>

In [57]:
def decide_winner(board,current_player):
    black = 0
    white = 0
    for x in board:
        for y in x:
            black += (y == 2)
            white += (y == 1)
    if(black + white != 64) and current_player == 2:
        return 0
    if(black + white != 64) and current_player == 1:
        return 1
    return int(black >= 32)

In [58]:
from copy import deepcopy
import random

def FLATTEN(arr):
    ret = []
    for x in arr: ret = ret + x
    return ret

def naiveValue(board):
    val = 0
    for x in board:
        for y in x:
            if y == 2: val += 1
            if y == 1: val -= 1
    return val

def minimax(board,depth,alpha,beta,maximizingPlayer,current_player):
   
    valid_moves = []
    for x in range(GRID_SIZE):
        for y in range(GRID_SIZE):
            if is_valid_move(x,y,board,current_player): valid_moves.append((x,y))
    
    if depth == 0: 
        with torch.no_grad():
            val = OthelloNet(torch.tensor(FLATTEN(board),dtype=torch.float32,device = device))
        return val
        # return naiveValue(board)
    
    if len(valid_moves) == 0: return decide_winner(board,current_player) * 64

    if maximizingPlayer:
        value = -100
        for x,y in valid_moves:
          
            child = deepcopy(board)
            make_move(x,y,child,2)
           
            value = max(value,minimax(child,depth-1,alpha,beta,False,1))
            if value > beta:
                break
            alpha = max(alpha,value)
        return value
    else:
        value = 100
        for x,y in valid_moves:
            
            child = deepcopy(board)
            make_move(x,y,child,1)
        
            value = min(value,minimax(child,depth-1,alpha,beta,True,2))
            if value < alpha:
                break
            beta = min(beta,value)
        return value


def get_move(board):
    valid_moves = []
    for x in range(GRID_SIZE):
        for y in range(GRID_SIZE):
            if is_valid_move(x,y,board,2): valid_moves.append((x,y))
    
    points = []
    for x,y in valid_moves:
  
        sim = deepcopy(board)
        make_move(x,y,sim,2)
        
        points.append(minimax(sim,1,-1000,+1000,False,1))

    maxi = 0
    for i in range(len(valid_moves)):
        if points[i] > points[maxi]:
            maxi = i

    return valid_moves[maxi]

def get_move_random(board,current_player):
    valid_moves = []
    for x in range(GRID_SIZE):
        for y in range(GRID_SIZE):
            if is_valid_move(x,y,board,current_player): valid_moves.append((x,y))
    return valid_moves[random.randint(0,len(valid_moves)-1)]


In [59]:
# Game loop
import time
games = 0
wins = 0
max_games = 50

current_player = 2
while games < max_games:
    # time.sleep(1)

    Quit = False
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            Quit = True
    if(Quit): break
       
    flag = 0
    for x in range(GRID_SIZE):
        for y in range(GRID_SIZE):
            if is_valid_move(x,y,board,current_player): flag=1

    if current_player == 1 and flag:
        x,y = get_move_random(board,current_player)    
        if is_valid_move(x, y,board,current_player):
            make_move(x,y,board,current_player)
        current_player = 2
       
    elif current_player == 2 and flag:
 
        grid_x,grid_y = get_move(board)
    
        if is_valid_move(grid_x, grid_y,board,current_player):
            make_move(grid_x, grid_y,board,current_player)
        current_player = 1

    if not flag:
        print(games)
        wins += decide_winner(board,current_player)
        board = [[0 for _ in range(GRID_SIZE)] for _ in range(GRID_SIZE)]
        board[3][4] = board[4][3] = 2  # White
        board[3][3] = board[4][4] = 1  # Black
        games += 1
        current_player = 2

    draw_board()
    pygame.display.flip()
    
pygame.quit()
print("wins ",wins)
    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
wins  41


2 minimax OthelloNet (3 moves ahead) - 43/50 against random<br>
2 minimax Naive Value (3 moves ahead) - 41/50 against random # Faster too<br>
4 minimax Naive Value - 45/50 agianst random